# Phyling API

To use this notebook, you need to create a `.env` file with:

```env
PHYLING_API_KEY=your_api_key

# Server API:   https://api.app.phyling.fr
# Maxi-Hub API: 192.168.1.42:5001
PHYLING_API_URL="https://api.app.phyling.fr"
```

In [1]:
import json
import time
import logging
import os
from dotenv import load_dotenv

from phyling.api import PhylingAPI
from phyling.api import PhylingRealtime

load_dotenv()
logging.getLogger().setLevel(logging.INFO)

# API connection

In [2]:
# Create API and Realtime instances
api = PhylingAPI(
    api_key=os.getenv("PHYLING_API_KEY"),
    url=os.getenv("PHYLING_API_URL", "https://api.app.phyling.fr/")
)

realtime = PhylingRealtime(api)

INFO:root:[API_CALL] 0.06s - GET http://localhost:5001/login - 200
INFO:root:PhylingAPI(baseurl=http://localhost:5001) -> Connected as tim.nicolas@phyling.fr
INFO:root:Socket connected to http://localhost:5001


INFO:root:Socket message info: Joined room: app/client/2/device/list_connected
INFO:root:Socket message info: Joined room: app/device/10300378/board/status
INFO:root:Socket message info: Joined room: app/device/10300378/ind/json/all
INFO:root:Socket message info: Joined room: app/device/10300378/data/json/all


In [3]:
# Start auto-updating device list every socketIO update
realtime.autoUpdateDeviceList(True)

INFO:root:[API_CALL] 0.02s - POST http://localhost:5001/devices/rt/2/all - 200


In [4]:
# Print the device list (will update automatically in the background)
print(realtime.getDeviceList())

[{'number': 10300378, 'name': 'Maxi 378', 'is_connected': True, 'state': 'idle'}]


In [5]:
# Enable auto-updates for device status, indicator, and data for the first connected device
number = realtime.getDeviceList()[0]["number"]
realtime.autoUpdateDeviceStatus(number=number, enabled=True)
realtime.autoUpdateDeviceIndicator(number=number, enabled=True)
realtime.autoUpdateDeviceData(number=number, enabled=True)

INFO:root:[API_CALL] 0.03s - GET http://localhost:5001/devices/rt/2/10300378/settings - 200
INFO:root:Initial settings for device 10300378: {'battery': 35, 'client': {'id': 2, 'name': 'phyling'}, 'deviceRecId': -1, 'epoch': 1763633850.0, 'features': {'calib': {'cmd_get': 'v1.board.calib.get', 'cmd_set': 'v1.board.calib.set'}, 'config': {'cmd_get': 'v1.board.config.get', 'cmd_set': 'v1.board.config.set'}, 'device_info': {'cmd_get': 'v1.board.device_info.get', 'cmd_set': 'v1.board.device_info.set'}, 'firmware_update': {'cmd': 'v1.board.firmware_update'}, 'logs': {'cmd_get': 'v1.logs.get', 'cmd_list': 'v1.logs.list'}, 'record': {'cmd_start': 'v1.record.rec.start', 'cmd_stop': 'v1.record.rec.stop'}, 'restart': {'cmd': 'v1.board.restart'}, 'scenario': {'cmd_start': 'v1.record.scenario.start', 'cmd_stop': 'v1.record.scenario.stop'}, 'shutdown': {'cmd': 'v1.board.shutdown'}, 'time_set': {'cmd': 'v1.board.time.set'}, 'wifi': {'cmd_get': 'v1.board.wifi.get', 'cmd_set': 'v1.board.wifi.set'}}, 'g

In [6]:
# Print the device status (will update automatically in the background)
print(realtime.getDeviceStatus(number=number))

{'battery': 35, 'client': {'id': 2, 'name': 'phyling'}, 'deviceRecId': -1, 'epoch': 1763633850.0, 'features': {'calib': {'cmd_get': 'v1.board.calib.get', 'cmd_set': 'v1.board.calib.set'}, 'config': {'cmd_get': 'v1.board.config.get', 'cmd_set': 'v1.board.config.set'}, 'device_info': {'cmd_get': 'v1.board.device_info.get', 'cmd_set': 'v1.board.device_info.set'}, 'firmware_update': {'cmd': 'v1.board.firmware_update'}, 'logs': {'cmd_get': 'v1.logs.get', 'cmd_list': 'v1.logs.list'}, 'record': {'cmd_start': 'v1.record.rec.start', 'cmd_stop': 'v1.record.rec.stop'}, 'restart': {'cmd': 'v1.board.restart'}, 'scenario': {'cmd_start': 'v1.record.scenario.start', 'cmd_stop': 'v1.record.scenario.stop'}, 'shutdown': {'cmd': 'v1.board.shutdown'}, 'time_set': {'cmd': 'v1.board.time.set'}, 'wifi': {'cmd_get': 'v1.board.wifi.get', 'cmd_set': 'v1.board.wifi.set'}}, 'group': {'id': 1, 'name': 'phyling'}, 'is_connected': True, 'modules': {'gps': {'id': 1, 'is_connected': False, 'type': 'gps'}, 'imu': {'id':

In [7]:
# Print the device state only
print(realtime.getDeviceStatus(number=number)["state"])

idle


In [8]:
# Print the available RPC methods
print(realtime.getDeviceStatus(number=number)["features"])

{'calib': {'cmd_get': 'v1.board.calib.get', 'cmd_set': 'v1.board.calib.set'}, 'config': {'cmd_get': 'v1.board.config.get', 'cmd_set': 'v1.board.config.set'}, 'device_info': {'cmd_get': 'v1.board.device_info.get', 'cmd_set': 'v1.board.device_info.set'}, 'firmware_update': {'cmd': 'v1.board.firmware_update'}, 'logs': {'cmd_get': 'v1.logs.get', 'cmd_list': 'v1.logs.list'}, 'record': {'cmd_start': 'v1.record.rec.start', 'cmd_stop': 'v1.record.rec.stop'}, 'restart': {'cmd': 'v1.board.restart'}, 'scenario': {'cmd_start': 'v1.record.scenario.start', 'cmd_stop': 'v1.record.scenario.stop'}, 'shutdown': {'cmd': 'v1.board.shutdown'}, 'time_set': {'cmd': 'v1.board.time.set'}, 'wifi': {'cmd_get': 'v1.board.wifi.get', 'cmd_set': 'v1.board.wifi.set'}}


In [9]:
# Perform a RPC request
realtime.executeRPC(
    number=number,
    method="v1.record.rec.start",
    params={}
)

INFO:root:[API_CALL] 0.03s - POST http://localhost:5001/devices/rt/2/10300378/rpc/request - 201


In [10]:
# Print the device indicator (will update automatically in the background - only on record/scenario states)
print(realtime.getDeviceIndicator(number=number))

{'number': 10300378, 'recTime': -1, 'indicators': {}}


In [11]:
# Print the latest realtime data (will update automatically in the background - only on record/scenario states)
lastRtData = realtime.getDeviceData(number=number)
print(lastRtData)

{'number': 10300378, 'recTime': -1, 'data': {}, 'selections': []}


In [12]:
# Continuously print new realtime data as it arrives
ONLY_FIRST = True
lastRecTime = 0
while True:
    status = realtime.getDeviceStatus(number=number)
    rtData = realtime.getDeviceData(number=number)
    now = time.time()
    if rtData["recTime"] <= lastRecTime:
        continue
    lastRecTime = rtData["recTime"]
    dataStr = f"[{rtData['recTime']:6.3f}] "
    for modname, modcontent in rtData.get("data", {}).items():
        latency = now - status['startRecTimeSinceEpoch'] - rtData["data"][modname]["T"][-1]
        dataStr +=  f" | latency: {int(latency * 1000):4d}ms"
        for key, value in modcontent.items():
            if key in ["T"]:
                continue
            if len(dataStr) > 0:
                dataStr += " | "
            dataStr += f"{modname}.{key}: {value[-1]:.2f}"
            if ONLY_FIRST:
                break
    if len(dataStr) > 0:
        print(dataStr)
    time.sleep(0.01)

KeyboardInterrupt: 

In [13]:
# Continuously print new realtime indicators as they arrive
# {'number': 10200300, 'recTime': 178.123, 'indicators': {'acc_x': {'x': 177.842491, 'y': 0.2968113739043474}}}
ONLY_FIRST = False
lastRecTime = 0
while True:
    status = realtime.getDeviceStatus(number=number)
    rtIndicators = realtime.getDeviceIndicator(number=number)
    now = time.time()
    if rtIndicators["recTime"] <= lastRecTime:
        continue
    lastRecTime = rtIndicators["recTime"]
    dataStr = f"[{rtIndicators['recTime']:6.3f}] "
    for indname, indcontent in rtIndicators.get("indicators", {}).items():
        latency = now - status['startRecTimeSinceEpoch'] - rtIndicators["indicators"][indname]["x"]
        dataStr +=  f"latency: {int(latency * 1000):4d}ms {indname}: {indcontent['y']:.2f}"
        if ONLY_FIRST:
            break
    if len(dataStr) > 0:
        print(dataStr)
    time.sleep(0.01)

KeyboardInterrupt: 